In [1]:
import numpy as np
import scipy.integrate
import matplotlib.pyplot as plt
%matplotlib notebook
%load_ext autoreload
%autoreload 2

plt.style.use('ggplot')

#### Define the Lotka-Volterra System

In [2]:
def f(t: float, x: np.ndarray, p: np.ndarray) -> np.ndarray:
    """ r.h.s. of the ODE system. """
    alpha = p[0]
    beta  = p[1]
    gamma = p[2]
    delta = p[3]
    I     = x[0]
    B     = x[1]
    return np.array([alpha*I-beta*I*B, gamma*I*B-delta*B])

def fx(t: float, x: np.ndarray, p: np.ndarray) -> np.ndarray:
    """ Jacobian of f w.r.t. states x. """
    alpha = p[0]
    beta  = p[1]
    gamma = p[2]
    delta = p[3]
    I     = x[0]
    B     = x[1]
    # we return fx as matrix because we need to perform a matrix-matrix multiplication with it in f_vode
    return np.array([ [alpha - beta*B, -beta*I], [gamma*B, gamma*I-delta] ])

def fp(t: float, x: np.ndarray, p: np.ndarray) -> np.ndarray:
    """ Jacobian of f w.r.t. parameters p. """
    alpha = p[0]
    beta  = p[1]
    gamma = p[2]
    delta = p[3]
    I     = x[0]
    B     = x[1]
    return np.array([ [I, -I*B, 0., 0.], [0., 0., I*B, -B] ])

#### Write a function to solve the IVP and compute sensitivities

In [3]:
class IVPResult:
    # this is the most generic way of defining a class. In Python class object attributes do not necessarily have to
    # be defined previously (even though this is good practice). I.e. we can just add any attribute my_att to an 
    # object obj of this class by executing e.g. obj.my_att = 0. We do this for example with result.success
    
    # The advantage of using a class object to store the results of the integration is that solve_ivp only needs
    # to return result and we do not need to modify the return and the return handling of solve_ivp which is very
    # convenient in larger projects, where solve_ivp might be called in many different places.
    pass

def solve_ivp(f: callable, ts: np.ndarray, x0: np.ndarray, p: np.ndarray = None, integrator: str = 'dopri5', store_trajectory: bool = False,
              sens: bool = False, dfx: callable = None, dfp: callable = None):
    """
    Solve initial value problem
        d/dt x = f(t, x, p);  x(t0) = x0.

    Evaluate Solution at time points specified in ts, with ts[0] = t0.

    Compute sensitivity matrices evaluated at time points in ts if sens=True.
    """
    # Note that xGxGp is the concatenated array for x, Gx and Gp
    nx    = x0.shape[0] # number of states
    nps    = p.shape[0]  # number of parameters
    
    if sens:
        def f_vode(t, xGxGp, p):  
            """ r.h.s. of the system ODE concatenated with the r.h.s. of VDE for Gx and Gp. """

            # Note that functions that determine the r.h.s. of the ODE MUST have the "time" argument in the first
            # place, then the vector with the quanitites we are integrating (scipy internally this is x. We however
            # call it xGxGp as it contains x, Gx and Gp in our notation). It usually MUST NOT have further arguments.
            # However, an additional argument can be declared by the means of set_f_params -> see below            
            
            # decompose xGxGp into x Gx, Gp
            x     = xGxGp[:nx]
            Gx    = xGxGp[nx: nx + nx * nx]  # Gx is nx \times nx
            Gx    = Gx.reshape([nx, nx])
            Gp    = xGxGp[nx + nx * nx:]  # Gp is nx \times nps
            Gp    = Gp.reshape([nx, nps])
            
            # evaluate f, dfx, dfp to get the problem dependent expression in the r.h.s of the ODE 
            dx    = f(t, x, p)
            dfxev = dfx(t, x, p)
            dfpev = dfp(t, x, p)
            
            # multiply dfxex, dfpev with Gx and Gp to formulate the r.h.s of the ODEs for Gx and Gp (r.h.s of x is dx)
            dGx   = dfxev.dot(Gx)
            dGp   = dfxev.dot(Gp) + dfpev
            
            # flatten and concatenate all parts of the r.h.s. -> scipy.integrate only handles a SINGLE VECTOR for the r.h.s
            return np.concatenate(
                [dx,
                 dGx.reshape(-1), 
                 dGp.reshape(-1)])

        ivp = scipy.integrate.ode(f_vode)  # tell scipy.integrate to use f_vode to compute the r.h.s.
        
    else:
        ivp = scipy.integrate.ode(f)  # if we don't need the sensitivity matrices, use only f for the r.h.s.

    ivp.set_integrator(integrator) 
    
    # possibly define a function that is called by scipy.integrate in every iteration. This allows us to extract
    # some more information. We are interested in the time steps scipy used for the integration and the values of
    # x at these points for plotting. Note that scipy.integrate would only return us the values at the time points
    # specified in ts and note that scipy.integrate chooses the integration step size on its own.
    if store_trajectory:  
        times = []
        points = []
        Gxs, Gps = [], []
        def solout(t, x):
            if len(times) == 0 or t != times[-1]:
                times.append(t)
                # x is scipy.integrate internally our xGxGp
                points.append(np.copy(x[:nx]))  
                if sens:
                    Gxs.append(np.copy(x[nx:nx+nx*nx].reshape([nx, nx])))
                    Gps.append(np.copy(x[nx+nx*nx:].reshape([nx, nps])))
        ivp.set_solout(solout)  # inform scipy to call solout after each integration

    if sens:  # set initial (flattened) array for x, Gx, Gp
        ivp.set_initial_value(np.concatenate(
            [x0,
             np.eye(nx).reshape(-1), # Gx=I; reshape(-1) flattens a matrix row-wise
             np.zeros([nx, nps]).reshape(-1)  # Gp=0
            ]), ts[0])  # ts[0] is the initial starting time -> that plays a role if t occurs explicitly in f
    else:
        ivp.set_initial_value(x0, ts[0])
        
    ivp.set_f_params(p)  # declare p an additional argument for the function that returns the r.h.s of the ODE (see f_vode for more details)

    result = IVPResult() # initialize empty IVPResult-object -> see Class definition for more information
    result.ts = ts
    nts = ts.shape[0]
    
    result.xs  = np.zeros([nts, nx])
    if sens:
        # We use multidimensional arrays to store the Gxs and Gps. This way we can store them as matrices and do not
        # have to flatten them again. E.g. result.Gxs[0] is a nx times nx matrix for Gx at ts[0]
        result.Gxs = np.zeros([nts, nx, nx])  
        result.Gps = np.zeros([nts, nx, nps])
    result.success = True

    result.xs[0,:] = x0
    # now we integrate until the points specified in ts. (This can be helpful if we want to know the result
    # at specific time points. scipy.integrate determines the integration step size on its own, i.e. if we would
    # not use this approach we might not be able to get a result at the time points in ts, because they might not
    # be covered by the time steps scipy.integrate uses within).
    for ii in range(1,nts):
        ivp.integrate(ts[ii])  # here the actual integration takes place
        result.xs[ii,:]    = ivp.y[:nx]
        if sens:
            result.Gxs[ii,:,:] = ivp.y[nx:nx+nx*nx].reshape([nx, nx])
            result.Gps[ii,:,:] = ivp.y[nx+nx*nx:].reshape([nx, nps])
        if not ivp.successful():
            result.success = False
            break

    if store_trajectory:
        result.trajectory_t = np.array(times)
        result.trajectory_x = np.array(points)
        if sens:
            result.trajectory_Gx = np.array(Gxs)
            result.trajectory_Gp = np.array(Gps)

    return result

#### (a) Solve and visualize the Lotka-Volterra system

In [4]:
x0 = np.array([20., 10.])
p = np.array([.2, .01, .001, .1])

ts = np.linspace(0., 500., 501) # time steps where the integrator is forced to stop
solution = solve_ivp(f, ts, x0, p, store_trajectory=True, sens=True, dfx=fx, dfp=fp)

plt.figure()
plt.plot(solution.trajectory_t, solution.trajectory_x)
plt.xlabel('Time')
plt.ylabel('Population')
plt.title('Lotka-Volterra-System')
plt.legend(['Ibex', 'Bears'])
plt.show()

<IPython.core.display.Javascript object>

#### (b) Compute the sensitivities $G^x(300)$ and $G^p(300)$

In [5]:
where_300 = np.where(solution.trajectory_t == 300)[0][0]
print("VDE-based Approximation of Gx:")
print(solution.trajectory_Gx[where_300, :, :])
print("VDE-based Approximation of Gp:")
print(solution.trajectory_Gp[where_300, :, :])

VDE-based Approximation of Gx:
[[ 0.72827851  1.69445434]
 [-2.47566257 -4.09254111]]
VDE-based Approximation of Gp:
[[-3.47683886e+01  1.69445434e+03  1.91676009e+03  1.83895906e+02]
 [-9.91562348e+02 -6.01270214e+03 -4.95132515e+04 -1.95233377e+03]]


#### (c) Comparison with finite differences

In [6]:
Gx_fd = np.zeros([x0.shape[0], x0.shape[0]])
Gp_fd = np.zeros([x0.shape[0], p.shape[0]])

# we need to get the nominal result at t=300
nominal_result = solution.trajectory_x[where_300, :]

h = 1e-3

# we disturb the initial values x0 and the parameter in each coordinate direction to get the columns of the
# approximations of Gx and Gp. As we wish to differentiate x, i.e. the states, w.r.t. x_0 and p, we always
# have to solve the ODE to get x(t_1, x_0, p)
e = np.zeros_like(x0)
for ii in range(x0.shape[0]):
    e[ii] = h * np.abs(x0[ii]) # scaled unit vector
    result = solve_ivp(f, ts=ts, x0=x0+e, p=p, store_trajectory=True)
    _where_300 = np.where(result.trajectory_t == 300)[0][0]
    Gx_fd[:, ii] = (result.trajectory_x[_where_300, :] - nominal_result) / e[ii]
    e[ii] = 0.
    
e = np.zeros_like(p)
for ii in range(p.shape[0]):
    e[ii] = h * np.abs(p[ii])
    result = solve_ivp(f, ts=ts, x0=x0, p=p+e, store_trajectory=True)
    _where_300 = np.where(result.trajectory_t == 300)[0][0]    
    Gp_fd[:, ii] = (result.trajectory_x[_where_300, :] - nominal_result) / e[ii]
    e[ii] = 0.
    
print("Finite Difference Approximation of Gx:")
print(Gx_fd)
print("Finite Difference Approximation of Gp:")
print(Gp_fd)
print("Infinity norm of the difference of the FD approximation and the VDE-based Gx and Gp, resp.:")
Gx_diff = np.linalg.norm(solution.trajectory_Gx[where_300, :, :] - Gx_fd, np.inf)
Gp_diff = np.linalg.norm(solution.trajectory_Gp[where_300, :, :] - Gp_fd, np.inf)
print(Gx_diff, Gp_diff)

Finite Difference Approximation of Gx:
[[ 0.72996748  1.69424534]
 [-2.46732628 -4.0755602 ]]
Finite Difference Approximation of Gp:
[[-2.62432658e+01  1.69424534e+03  1.94859099e+03  1.97599842e+02]
 [-9.85189828e+02 -5.98973150e+03 -4.93465257e+04 -1.94204963e+03]]
Infinity norm of the difference of the FD approximation and the VDE-based Gx and Gp, resp.:
0.025317197970590843 206.35309487136738
